# Imports

In [1]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

2025-04-26 19:44:59.138323: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-26 19:44:59.497849: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data Preparation

## Directories

Here we set up BASE_DIR, train_dir, validation_dir, img_height, img_width, batch_size, etc

In [4]:
'''
Create variables for the base directory where the data is located, followed by training and validation directories, followed by the class directories 
'''

# Directory that holds the data
BASE_DIR = '__________________'

# Training directory
train_dir = os.path.join(BASE_DIR, 'train')
# Validation directory
validation_dir = os.path.join(BASE_DIR, 'validation')

# Directory with training pictures
train__________________ = os.path.join(train_dir, '__________________')
train__________________ = os.path.join(train_dir, 'do__________________gs')

# Directory with validation pictures
validation__________________ = os.path.join(validation_dir, '__________________')
validation__________________ = os.path.join(validation_dir, '__________________')

# Check the contents of the directories
print(f"Contents of base directory: {os.listdir(BASE_DIR)}")

print(f"\nContents of train directory: {os.listdir(train_dir)}")

print(f"\nContents of validation directory: {os.listdir(validation_dir)}")

FileNotFoundError: [Errno 2] No such file or directory: '__________________'

## Contants & Data Loading

In [ ]:
# Constants
BATCH_SIZE = 32
IMAGE_SIZE = (300, 300)
LABEL_MODE = 'binary'

# Instantiate the training dataset
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode=LABEL_MODE
)

# Instantiate the validation dataset
validation_dataset = tf.keras.utils.image_dataset_from_directory(
    validation_dir,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode=LABEL_MODE
)
# Optional: Inspect the dataset here if desired
# print(train_dataset.class_names) # See inferred class names
# for image_batch, label_batch in train_dataset.take(1):
#     print(image_batch.shape)
#     print(label_batch.shape)

## Data Pipeline Optimization: Apply .cache(), .shuffle(), .prefetch()

In [5]:
SHUFFLE_BUFFER_SIZE = 1000
PREFETCH_BUFFER_SIZE = tf.data.AUTOTUNE

train_dataset_final = (train_dataset
                       .cache()
                       .shuffle(SHUFFLE_BUFFER_SIZE)
                       .prefetch(PREFETCH_BUFFER_SIZE)
                       )

validation_dataset_final = (validation_dataset
                            .cache()
                            .prefetch(PREFETCH_BUFFER_SIZE)
                            )

NameError: name 'train_dataset' is not defined

# Create Model Function

In [ ]:
""""
Builds a CNN for image binary classification;
We are defining a base model function, in case we have to create multiple variations and/or with augmentations before compiling/training models
"""

def create_model(): 
    model = tf.keras.models.Sequential([
      tf.keras.Input(shape=(300,300,3)),
      # This will rescale the image to [0,1]
      tf.keras.layers.Rescaling(1./255),
      # This is the first convolution
      tf.keras.layers.Conv2D(16, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      # The second convolution
      tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      # The third convolution
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      # The fourth convolution
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      # The fifth convolution
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      # Flatten the results to feed into a DNN
      tf.keras.layers.Flatten(),
      # 512 neuron hidden layer
      tf.keras.layers.Dense(512, activation='relu'),
      # Last 1 Neuron
      tf.keras.layers.Dense(1, activation='sigmoid')
      ])
    
    return model

In [ ]:
model.summary()

## Image Augmentation

In [ ]:
# Define fill mode.
FILL_MODE = 'nearest'

# Create the augmentation model.
data_augmentation = tf.keras.Sequential([
    # Specify the input shape.
    tf.keras.Input(shape=(150,150,3)),
    # Add the augmentation layers
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2, fill_mode=FILL_MODE),
    tf.keras.layers.RandomTranslation(0.2,0.2, fill_mode=FILL_MODE),
    tf.keras.layers.RandomZoom(0.2, fill_mode=FILL_MODE)
    ])

### Utility Function

In [ ]:
"""
 Utility function that lets you preview how the transformed images look like. 
 It will take in a sample image, then output a given number of augmented images using the model defined above.
"""

def demo_augmentation(sample_image, model, num_aug):
    '''Takes a single image array, then uses a model to generate num_aug transformations'''

    # Instantiate preview list
    image_preview = []

    # Convert input image to a PIL image instance
    sample_image_pil = tf.keras.utils.array_to_img(sample_image)

    # Append the result to the list
    image_preview.append(sample_image_pil)

    # Apply the image augmentation and append the results to the list
    for i in range(NUM_AUG):
        sample_image_aug = model(tf.expand_dims(sample_image, axis=0))
        sample_image_aug_pil = tf.keras.utils.array_to_img(tf.squeeze(sample_image_aug))
        image_preview.append(sample_image_aug_pil)

    # Instantiate a subplot
    fig, axes = plt.subplots(1, NUM_AUG + 1, figsize=(12, 12))

    # Preview the images.
    for index, ax in enumerate(axes):
        ax.imshow(image_preview[index])
        ax.set_axis_off()

        if index == 0:
            ax.set_title('original')
        else:
            ax.set_title(f'augment {index}')

In [ ]:
# Get a batch of images
sample_batch = list(train_dataset.take(1))[0][0]
print(f'images per batch: {len(sample_batch)}')

In [ ]:
NUM_AUG = 4

# Apply the transformations to the first 4 images
demo_augmentation(sample_batch[0], data_augmentation, NUM_AUG)
demo_augmentation(sample_batch[1], data_augmentation, NUM_AUG)
demo_augmentation(sample_batch[2], data_augmentation, NUM_AUG)
demo_augmentation(sample_batch[3], data_augmentation, NUM_AUG)

# Uncomment the line below to delete the variable to free up some memory
# del sample_batch

## Create Models

In [ ]:
# Instantiate the base model
model = create_model()
model2 = create_model()

# Prepend the data augmentation layers to the base model
model_with_aug = tf.keras.models.Sequential([
    data_augmentation,
    model2
])

# Any more augmentations? 


# Compile Model

In [ ]:
# compile the model
model.compile( 
        optimizer=tf.keras.optimizers.RMSprop(),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=['accuracy']
)

## variants

In [ ]:
# Compile the model
model_with_aug.compile(
    loss='binary_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4),
    metrics=['accuracy'])

# EarlyStoppingCallback

In [ ]:
# 
class EarlyStoppingCallback(tf.keras.callbacks.Callback):

    # Define the correct function signature for on_epoch_end method
    def on_epoch_end(self, epoch, logs=None):
        # Check if the accuracy is greater or equal to 0.95 and validation accuracy is greater or equal to 0.8
        if logs['accuracy'] >= 0.95 and logs['val_accuracy'] >= 0.80:
            print("\nReached 95% train accuracy and 80% validation accuracy, so cancelling training!")
            self.model.stop_training = True

# Train Model

In [ ]:
# Train the model and save the training history (this may take some time)
history = model.fit(
	train_dataset_final,
	epochs=15,
	validation_data=validation_dataset_final,
	callbacks = [EarlyStoppingCallback()]
)

## variants

In [ ]:
# Train the model and save the training history (this may take some time)
history_with_aug = model_with_aug.fit(
	train_dataset_final,
	epochs=15,
	validation_data=validation_dataset_final,
	callbacks = [EarlyStoppingCallback()]
)

# Plot Training and Validation Accuracy

In [ ]:
def plot_loss_acc(history):
    """Plots the training and validation loss and accuracy from a history object"""
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Get number of epochs
    epochs = range(len(acc))

    fig, ax = plt.subplots(1, 2, figsize=(10, 5))
    fig.suptitle('Training and validation accuracy')

    for i, (data, label) in enumerate(zip([(acc, val_acc), (loss, val_loss)], ["Accuracy", "Loss"])):
        ax[i].plot(epochs, data[0], 'r', label="Training " + label)
        ax[i].plot(epochs, data[1], 'b', label="Validation " + label)
        ax[i].legend()
        ax[i].set_xlabel('epochs')

    plt.show()

NameError: name 'history' is not defined

In [ ]:
plot_loss_acc(history)

In [ ]:
plot_loss_acc(history_with_aug)